In [1]:
import os
import numpy as np
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.nn.modules.conv as conv
import torch.utils.data as utils
import torch.nn.functional as F
from torchsummary import summary
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from coordconv import AddCoords
from models.CornerNet import model

# m = model()
# x = torch.rand(2, 3, 64, 64)
# m(x)

In [44]:
from torchsummary import summary

class OnehotNet(nn.Module):
    def __init__(self):
        super(OnehotNet, self).__init__()
        # self.coordconv = CoordConv2d(2, 32, 1, with_r=True)
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 1, 3, padding=1)
        self.conv4 = nn.Conv2d(1, 1, 1)

    def forward(self, x):
        # x = self.coordconv(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = x.view(-1, 64 * 64)
        return x
m = OnehotNet()
summary(m, (1, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 64, 64]              80
            Conv2d-2           [-1, 16, 64, 64]           1,168
            Conv2d-3            [-1, 1, 64, 64]             145
            Conv2d-4            [-1, 1, 64, 64]               2
Total params: 1,395
Trainable params: 1,395
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.81
Params size (MB): 0.01
Estimated Total Size (MB): 0.83
----------------------------------------------------------------
